In [3]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [4]:
df = dd.read_csv('NIS_2012_CoreCSV.csv', dtype=object)

List of columns with mixed types
21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,62,63,64,65,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104

In [21]:
df.head()

,AGE,AGE_NEONATE,AMONTH,AWEEKEND,DIED,DISCWT,DISPUNIFORM,DQTR,DRG,DRG24,...,PRDAY12,PRDAY13,PRDAY14,PRDAY15,RACE,TOTCHG,TRAN_IN,TRAN_OUT,YEAR,ZIPINC_QRTL
0,24,NaN,1,0,0,4.9999028,1,1,765,371,...,NaN,NaN,NaN,NaN,1,7620,0,0,2012,3
1,0,1,11,0,0,4.9999028,1,4,794,390,...,NaN,NaN,NaN,NaN,3,3551,0,0,2012,1
2,0,1,6,0,0,4.9999028,1,2,794,390,...,NaN,NaN,NaN,NaN,1,2794,0,0,2012,3
3,79,NaN,3,0,0,4.9999028,1,1,287,125,...,NaN,NaN,NaN,NaN,1,30402,0,0,2012,4
4,55,NaN,5,1,0,4.9999028,7,2,948,464,...,NaN,NaN,NaN,NaN,1,8083,0,0,2012,2


### List of HCUP column names:

In [5]:
len(df.DX1.unique())

9360

In [26]:
len(df[df['DX1']=='64814']) ##checks column for specified value

23

In [7]:
len(df[df['DX1']=='44389']) ##checks column for peripheral vascular disease

268

### Number of unique variables by column

In [8]:
for i in range(15):
    print("PR{}: ".format(i+1), len(df['PR{}'.format(i+1)].unique()))

PR1:  3325
PR2:  3327
PR3:  3204
PR4:  2989
PR5:  2783
PR6:  2529
PR7:  2258
PR8:  2038
PR9:  1819
PR10:  1629
PR11:  1369
PR12:  1277
PR13:  1189
PR14:  1024
PR15:  1020


In [9]:
for i in range(25):
    print("DX{}: ".format(i+1), len(df['DX{}'.format(i+1)].unique()))

DX1:  9360
DX2:  9999
DX3:  9938
DX4:  9637
DX5:  9379
DX6:  9143
DX7:  8928
DX8:  8725
DX9:  8424
DX10:  8115
DX11:  7824
DX12:  7624
DX13:  7397
DX14:  6999
DX15:  6995
DX16:  6119
DX17:  5742
DX18:  5453
DX19:  4934
DX20:  4554
DX21:  4287
DX22:  4056
DX23:  3829
DX24:  3623
DX25:  3320


In [10]:
len(df.DX3.unique())

9938

### Procedure Matrix

The following parser takes the Procedure codes and their descriptions as elements in the 'PRcol_heads' array.

In [13]:
PRcol_heads = []
with open("Procedures.txt", "r") as f:
    for line in f:
        code, labels = line.split("=")[0][4:-2], line.split("=")[1][2:-2]
        PRcol_heads.append((code,labels))
PRcol_heads[:10]

[('    ', '    : BLANK'),
 ('0001', '0001: THERAP ULTRASOUND OF HEAD AND NECK (Begin 2002)'),
 ('0002', '0002: THERAPEUTIC ULTRASOUND OF HEART (Begin 2002)'),
 ('0003', '0003: THERAP ULTRASOUND PERIPHRL VASC VESSELS (Begin 2002)'),
 ('0009', '0009: OTHER THERAPEUTIC ULTRASOUND (Begin 2002)'),
 ('0010', '0010: IMPLANTATION OF CHEMOTHERAPEUTIC AGENT (Begin 2002)'),
 ('0011', '0011: INFUSION DROTRECOGIN ALFA (ACTIVATED) (Begin 2002)'),
 ('0012', '0012: ADMINISTRATION OF INHALED NITRIC OXIDE (Begin 2002)'),
 ('0013', '0013: INJECTION OR INFUSION OF NESIRITIDE (Begin 2002)'),
 ('0014', '0014: INJECT/INFUS OF OXAZOLIDINONE ANTIBTCS (Begin 2002)')]

In [14]:
PRdict = dict(PRcol_heads) # converts the key value pairs to a dictionary

In [15]:
PRdict

{'    ': '    : BLANK',
 '0001': '0001: THERAP ULTRASOUND OF HEAD AND NECK (Begin 2002)',
 '0002': '0002: THERAPEUTIC ULTRASOUND OF HEART (Begin 2002)',
 '0003': '0003: THERAP ULTRASOUND PERIPHRL VASC VESSELS (Begin 2002)',
 '0009': '0009: OTHER THERAPEUTIC ULTRASOUND (Begin 2002)',
 '0010': '0010: IMPLANTATION OF CHEMOTHERAPEUTIC AGENT (Begin 2002)',
 '0011': '0011: INFUSION DROTRECOGIN ALFA (ACTIVATED) (Begin 2002)',
 '0012': '0012: ADMINISTRATION OF INHALED NITRIC OXIDE (Begin 2002)',
 '0013': '0013: INJECTION OR INFUSION OF NESIRITIDE (Begin 2002)',
 '0014': '0014: INJECT/INFUS OF OXAZOLIDINONE ANTIBTCS (Begin 2002)',
 '0015': '0015: HIGH-DOSE INFUSION INTERLEUKIN-2 (IL-2) (Begin 2003)',
 '0016': '0016: PRESSURIZED TREAT GRAFT (Begin 2004)',
 '0017': '0017: INFUSION OF VASOPRESSOR (Begin 2004)',
 '0018': '0018: INFUS IMMUNOSUP ANTIBODY (Begin 2005)',
 '0019': '0019: BBBD VIA INFUSION (Begin 2007)',
 '0021': '0021: IVUS EXTRACRAN CEREB VES (Begin 2004)',
 '0022': '0022: IVUS INTRATH

In [16]:
Procedure_mtx = pd.DataFrame(columns = PRdict.keys()) #create procedure matrix

In [17]:
Procedure_mtx.head()

,,0001,0002,0003,0009,0010,0011,0012,0013,0014,...,9988,9991,9992,9993,9994,9995,9996,9997,9998,9999


#### Procedure Matrix Populator


In [79]:
# Generate list of procedure columns
prcols = []
for i in np.arange(0,15,1):
    prcols.append("PR{}".format(i+1))

In [80]:
# Create df of only 15 procedure code columns for simplicity
dfpx = df[prcols]

In [83]:
dfpx.head()

,PR1,PR2,PR3,PR4,PR5,PR6,PR7,PR8,PR9,PR10,PR11,PR12,PR13,PR14,PR15
0,741,7534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3722,8856,8853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
for row in dfpx.itertuples():
    print(row)
    break

Pandas(Index=0, PR1='741', PR2='7534', PR3=nan, PR4=nan, PR5=nan, PR6=nan, PR7=nan, PR8=nan, PR9=nan, PR10=nan, PR11=nan, PR12=nan, PR13=nan, PR14=nan, PR15=nan)


In [ ]:
for row in dfpx.row

### Diagnosis Matrix

The following parser takes the Diagnosis codes and their descriptions as elements in the 'DXcol_heads' array.

In [18]:
DXcol_heads = []
with open("Diagnosis.txt", "r") as f:
    for line in f:
        code, labels = line.split("=")[0][4:-2], line.split("=")[1][2:-2]
        code, labels = line.split("=")[0][4:-2], line.split("=")[1][2:-2]
        DXcol_heads.append((code,labels))
DXcol_heads[:10]

[('0010 ', '0010 : CHOLERA D/T VIB CHOLERAE'),
 ('0011 ', '0011 : CHOLERA D/T VIB EL TOR'),
 ('0019 ', '0019 : CHOLERA NOS'),
 ('0020 ', '0020 : TYPHOID FEVER'),
 ('0021 ', '0021 : PARATYPHOID FEVER A'),
 ('0022 ', '0022 : PARATYPHOID FEVER B'),
 ('0023 ', '0023 : PARATYPHOID FEVER C'),
 ('0029 ', '0029 : PARATYPHOID FEVER NOS'),
 ('0030 ', '0030 : SALMONELLA ENTERITIS'),
 ('0031 ', '0031 : SALMONELLA SEPTICEMIA')]

In [19]:
DXdict = dict(DXcol_heads)

In [20]:
DXdict

{'0010 ': '0010 : CHOLERA D/T VIB CHOLERAE',
 '0011 ': '0011 : CHOLERA D/T VIB EL TOR',
 '0019 ': '0019 : CHOLERA NOS',
 '0020 ': '0020 : TYPHOID FEVER',
 '0021 ': '0021 : PARATYPHOID FEVER A',
 '0022 ': '0022 : PARATYPHOID FEVER B',
 '0023 ': '0023 : PARATYPHOID FEVER C',
 '0029 ': '0029 : PARATYPHOID FEVER NOS',
 '0030 ': '0030 : SALMONELLA ENTERITIS',
 '0031 ': '0031 : SALMONELLA SEPTICEMIA',
 '00320': '00320: LOCAL SALMONELLA INF NOS',
 '00321': '00321: SALMONELLA MENINGITIS',
 '00322': '00322: SALMONELLA PNEUMONIA',
 '00323': '00323: SALMONELLA ARTHRITIS',
 '00324': '00324: SALMONELLA OSTEOMYELITIS',
 '00329': '00329: LOCAL SALMONELLA INF NEC',
 '0038 ': '0038 : SALMONELLA INFECTION NEC',
 '0039 ': '0039 : SALMONELLA INFECTION NOS',
 '0040 ': '0040 : SHIGELLA DYSENTERIAE',
 '0041 ': '0041 : SHIGELLA FLEXNERI',
 '0042 ': '0042 : SHIGELLA BOYDII',
 '0043 ': '0043 : SHIGELLA SONNEI',
 '0048 ': '0048 : SHIGELLA INFECTION NEC',
 '0049 ': '0049 : SHIGELLOSIS NOS',
 '0050 ': '0050 : STAP

In [21]:
Diagnosis_mtx = pd.DataFrame(columns = DXdict.keys()) #create diagnosis matrix

In [22]:
Diagnosis_mtx.head()

,0010,0011,0019,0020,0021,0022,0023,0029,0030,0031,...,V9112,V9119,V9120,V9121,V9122,V9129,V9190,V9191,V9192,V9199


### Attempt at dd.get_dummies

In [5]:
ddf_known = df.categorize() #change to only categorize DXi's and PRi's columns

In [6]:
ddf_known.DX10

Dask Series Structure:
npartitions=34
    category[known]
                ...
         ...       
                ...
                ...
Name: DX10, dtype: category
Dask Name: getitem, 170 tasks

In [11]:
Diag_mtx = dd.get_dummies(ddf_known["DX1"]) # get_dummies only works for one column
#error: Diag_mtx = dd.concat([dd.get_dummies(ddf_known[col]) for col in ddf_known], axis=1, keys=ddf_known.columns)

In [12]:
Diag_mtx.head()

DX1,66001,V3001,V3000,42731,33819,78097,56211,53021,6826,4255,...,81300,27902,80059,29525,9138,38522,80462,6981,7365,80164
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
exdf = pd.DataFrame({'ID': ['Sam', 'Alex', 'Sam'], 'A': ['a','b','c'], 'B': ['b','a','b'], 'C': ['c','c','a']})


In [68]:
exdf

,A,B,C,ID
0,a,b,c,Sam
1,b,a,c,Alex
2,c,b,a,Sam


In [61]:
pd.get_dummies(exdf, columns = ['A','B','C'], sparse=True) #to illustrate that unique identifiers stay paired with observations

,ID,A_a,A_b,A_c,B_a,B_b,C_a,C_c
0,Sam,1,0,0,0,1,0,1
1,Alex,0,1,0,1,0,0,1
2,Sam,0,0,1,0,1,1,0


In [71]:
for i,x in enumerate(exdf.columns):
        print(i,x)

0 A
1 B
2 C
3 ID
